[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MAROON-LABKOM/script-praktikum/blob/master/pemrograman-data-analisis/segmentasi-rfm.ipynb)

In [51]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [52]:
url = "https://github.com/MAROON-LABKOM/script-praktikum/blob/master/pemrograman-data-analisis/northwind_db.xlsx?raw=true"

df_products = pd.read_excel(url, sheet_name="products")
df_customers = pd.read_excel(url, sheet_name="customers")
df_orders = pd.read_excel(url, sheet_name="orders")
df_orders_details = pd.read_excel(url, sheet_name="order_details")

In [53]:
df_RFM = df_customers.merge(df_orders, left_on="customer_id", right_on="customer_id", how="inner")
df_RFM = df_RFM.merge(df_orders_details, left_on="order_id", right_on="order_id", how="inner")
df_RFM["order_date"] = pd.to_datetime(df_RFM["order_date"])

In [54]:
df_RFM.head()

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,...,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country,product_id,unit_price,quantity,discount
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NaN,12209,Germany,030-0074321,...,Alfreds Futterkiste,Obere Str. 57,Berlin,NaN,12209,Germany,28,456,15,25
1,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NaN,12209,Germany,030-0074321,...,Alfreds Futterkiste,Obere Str. 57,Berlin,NaN,12209,Germany,39,18,21,25
2,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NaN,12209,Germany,030-0074321,...,Alfreds Futterkiste,Obere Str. 57,Berlin,NaN,12209,Germany,46,12,2,25
3,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NaN,12209,Germany,030-0074321,...,Alfred's Futterkiste,Obere Str. 57,Berlin,NaN,12209,Germany,63,439,20,0
4,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NaN,12209,Germany,030-0074321,...,Alfred's Futterkiste,Obere Str. 57,Berlin,NaN,12209,Germany,3,10,6,0


## Segmentasi RFM

Setelah didapatkan tabel dengan semua informasi yang dibutuhkan,

 - banyaknya `customer_id` yang unik akan menjadi **frequency**
 - selisih tanggal antara `order_date` dan tanggal analisis akan menjadi **recency**
 - `unit_price`, `quantity`, dan `discount` akan menjadi faktor **monetary**

 Selanjutnya hitung komponen RFM menggunakan fungsi apply dengan melakukan agregasi pada kolom-kolom di atas.

In [55]:
# Recency
# Ambil selisih waktu antara waktu order (order_date) dan waktu referensi (1999-01-01)

date_ref = pd.to_datetime("1999-01-01")
df_features = pd.DataFrame(df_RFM.groupby("customer_id")["order_date"].max()).reset_index()
df_features["recency"] = df_features.apply(lambda x: date_ref - x["order_date"], axis=1)
df_features["recency"] = df_features["recency"].dt.days

df_features = df_features.drop("order_date", axis=1)

In [56]:
# Frequency
# Hitung banyaknya transaksi yang dilakukan oleh seseorang

df_temp_freq = pd.DataFrame(df_RFM.groupby("customer_id")["order_date"].count())
df_temp_freq = df_temp_freq.rename(columns={"order_date": "frequency"})
df_temp_freq = df_temp_freq.reset_index()

df_features = df_features.merge(df_temp_freq, on="customer_id")

In [57]:
# Monetary
# Hitung total pembelian pada suatu order
# (unit_price * quantity) - (unit_price * quantity * discount / 100.0)

df_temp_monetary = df_RFM.copy()
#df_temp_monetary["monetary_temp"] = (df_temp_monetary["unit_price"] * df_temp_monetary["quantity"]) - (df_temp_monetary["unit_price"] * df_temp_monetary["quantity"] * df_temp_monetary["discount"] / 100.0)
df_temp_monetary["total_price"] = df_temp_monetary["unit_price"] * df_temp_monetary["quantity"]

df_temp_monetary = pd.DataFrame(df_temp_monetary.groupby("customer_id")["total_price"].sum())
df_temp_monetary = df_temp_monetary.rename(columns={"total_price": "monetary"})
df_temp_monetary = df_temp_monetary.reset_index()

df_features = df_features.merge(df_temp_monetary, on="customer_id")

In [58]:
df_features.head()

,customer_id,recency,frequency,monetary
0,ALFKI,267,12,75815
1,ANATR,303,10,12279
2,ANTON,338,17,156617
3,AROUT,266,30,107555
4,BERGS,303,52,611515


In [59]:
df_features.describe().round(2)

,recency,frequency,monetary
count,89.00,89.00,89.00
mean,291.56,24.21,270989.58
std,87.48,19.79,487106.57
min,240.00,2.00,288.00
25%,248.00,11.00,35488.00
50%,263.00,20.00,99988.00
75%,298.00,31.00,293050.00
max,897.00,116.00,2646773.00


Setelah mendapatkan tiga komponen RFM, selanjutnya bisa dibuat box plot untuk menunjukkan empat kuartil pada data yang akan digunakan untuk melakukan segmentasi menggunakan kuartil.

In [60]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Recency", "Frequency", "Monetary"))

fig.add_trace(go.Box(y=df_features["recency"], boxpoints="all"), row=1, col=1)
fig.add_trace(go.Box(y=df_features["frequency"], boxpoints="all"), row=1, col=2)
fig.add_trace(go.Box(y=df_features["monetary"], boxpoints="all"), row=1, col=3)

fig.update_layout(showlegend=False)
fig.show()

## Konversi variabel RFM ke bin 1234

Berdasarkan kuartil yang didapatkan pada plot di atas, selanjutnya adalah proses mengubah komponen RFM yang berupa data kontinu menjadi data ordinal untuk menunjukkan tingkatan RFM berdasarkan kuartil.

In [61]:
def convert_R_score(recency):
  if (recency >= 0.0 and recency <= 248.0):
    return 4
  if (recency > 248.0 and recency <= 263.0):
    return 3
  if (recency > 263.0 and recency <= 298.0):
    return 2
  else:
    return 1

In [62]:
def convert_F_score(recency):
  if (recency >= 0.0 and recency <= 11.0):
    return 1
  if (recency > 11.0 and recency <= 20.0):
    return 2
  if (recency > 20.0 and recency <= 31.0):
    return 3
  else:
    return 4

In [63]:
def convert_M_score(recency):
  if (recency >= 0.0 and recency <= 3239.8000):
    return 1
  if (recency > 3239.8000 and recency <= 7048.2400):
    return 2
  if (recency > 7048.2400 and recency <= 16817.0975):
    return 3
  else:
    return 4

In [64]:
df_features["R_score"] = df_features.apply(lambda x: convert_R_score(x["recency"]), axis=1)
df_features["F_score"] = df_features.apply(lambda x: convert_F_score(x["frequency"]), axis=1)
df_features["M_score"] = df_features.apply(lambda x: convert_M_score(x["monetary"]), axis=1)

In [65]:
df_features.head()

,customer_id,recency,frequency,monetary,R_score,F_score,M_score
0,ALFKI,267,12,75815,2,2,4
1,ANATR,303,10,12279,1,1,3
2,ANTON,338,17,156617,1,2,4
3,AROUT,266,30,107555,2,3,4
4,BERGS,303,52,611515,1,4,4


Setelah didapatkan skor RFM yang merupakan data ordinal dari hasil kuartil, selanjutnya bisa dibuat tabel kontingensi RFM untuk menunjukkan berapa banyak customer yang masuk ke dalam kategori RFM tertentu.

In [66]:
# Tabel Kontingensi

customer_segment_contingency = pd.pivot_table(df_features, index=["F_score", "M_score"], columns="R_score", values="customer_id", aggfunc='count')
customer_segment_contingency.fillna(0)

R_score            1    2    3     4
F_score M_score                     
1       1        3.0  0.0  0.0   1.0
        2        1.0  0.0  0.0   0.0
        3        3.0  0.0  0.0   1.0
        4        5.0  8.0  2.0   2.0
2       3        0.0  1.0  0.0   0.0
        4        4.0  8.0  4.0   3.0
3       4        4.0  4.0  8.0   6.0
4       4        2.0  1.0  8.0  10.0

Selain tabel kontingensi, selanjutnya akan dibuat dua variabel baru yaitu:

- `customer_activity` menunjukkan apakah customer tersebut Active, At Risk, atau Churned berdasarkan `R_score`
- `customer_value` menunjukkan nilai customer berdasarkan `F_score` dan `M_score` untuk mengkategorikan pelanggan menjadi Premium, Gold, dan Silver.

In [77]:
def customer_value_map(F_score, M_score):
  if F_score == 4 and M_score == 4:
    return "Premium"
  elif (F_score >= 2 and F_score <= 4) and (M_score >= 2 and M_score <= 4):
    return "Gold"
  else:
    return "Silver"

seg_quantile = df_features.copy()
seg_quantile["customer_activity"] = seg_quantile["R_score"].map({4: 'Active', 3: 'At Risk', 2: 'Churned', 1: 'Churned'})
seg_quantile["customer_value"] = seg_quantile.apply(lambda x: customer_value_map(x["F_score"], x["M_score"]), axis=1)

In [78]:
seg_quantile.head()

,customer_id,recency,frequency,monetary,R_score,F_score,M_score,customer_activity,customer_value
0,ALFKI,267,12,75815,2,2,4,Churned,Gold
1,ANATR,303,10,12279,1,1,3,Churned,Silver
2,ANTON,338,17,156617,1,2,4,Churned,Gold
3,AROUT,266,30,107555,2,3,4,Churned,Gold
4,BERGS,303,52,611515,1,4,4,Churned,Premium
